In [1]:
import os, sys
sys.path.insert(0, '/clusterCAD')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "clusterCAD.settings")
import django
django.setup()
import pks.models
import compounddb.models

In [2]:
# save compounds in the database
caf = compounddb.models.Compound(name="Caffeine", smiles="O=C2N(c1ncn(c1C(=O)N2C)C)C")
# caf.computeInchi()
caf.save()

cw = compounddb.models.Compound(name="cw", smiles="CCCC1=CC=C(C=C1)C(C)C(=O)O")
# cw.computeInchi()
cw.save()

In [3]:
caf.inchiKey
# caf.delete()

'RYYVLZVUVIJVGH-UHFFFAOYSA-N'

In [4]:
# test substructure search
compounds = compounddb.models.Compound.objects.raw('select \"inchiKey\" from compounddb_compound where m@>\'CCCc1ccc(C(C)C(=O)O)cc1\';')
[x.name for x in compounds]

['cw']

In [9]:
results = compounddb.models.Compound.atomPairSearch('CCCC1=CC=C(C=C1)C(C)C(=O)O', maxHits=10, minSim=0)
[(result[0], result[1].mol()) for result in results]

[(1.0, <rdkit.Chem.rdchem.Mol at 0x7f3fbcd4d458>),
 (0.192090395480226, <rdkit.Chem.rdchem.Mol at 0x7f3fbcd4d528>),
 (0.172222222222222, <rdkit.Chem.rdchem.Mol at 0x7f3fbcd4d5f8>),
 (0.166666666666667, <rdkit.Chem.rdchem.Mol at 0x7f3fbcd4d660>),
 (0.165745856353591, <rdkit.Chem.rdchem.Mol at 0x7f3fbcde2ba8>),
 (0.159340659340659, <rdkit.Chem.rdchem.Mol at 0x7f3fbcde2b40>),
 (0.149659863945578, <rdkit.Chem.rdchem.Mol at 0x7f3fbcde29a0>),
 (0.149606299212598, <rdkit.Chem.rdchem.Mol at 0x7f3fbcde2800>),
 (0.146464646464646, <rdkit.Chem.rdchem.Mol at 0x7f3fbcde2528>),
 (0.145985401459854, <rdkit.Chem.rdchem.Mol at 0x7f3fbcde2388>)]

In [24]:
# find which gene clusters, subunit, and module each top hit is from
[[(result[0], module.subunit.cluster, module.subunit, module)
  for module in pks.models.Module.objects.filter(product=result[1])]
     for result in results]

[[],
 [(0.192090395480226,
   <Cluster: Divergolide pks gene cluster>,
   <Subunit: divL1 pks subunit>,
   <Module: pks module 1012>),
  (0.192090395480226,
   <Cluster: Hygrocin pks gene cluster>,
   <Subunit: hgcB pks subunit>,
   <Module: pks module 280>)],
 [(0.172222222222222,
   <Cluster: Naphthomycin pks gene cluster>,
   <Subunit: natB pks subunit>,
   <Module: pks module 459>),
  (0.172222222222222,
   <Cluster: Rifamycin pks gene cluster>,
   <Subunit: rifB pks subunit>,
   <Module: pks module 604>),
  (0.172222222222222,
   <Cluster: Chaxamycin A / chaxamycin B / chaxamycin C / chaxamycin D pks gene cluster>,
   <Subunit: cxmB pks subunit>,
   <Module: pks module 1040>)],
 [],
 [(0.165745856353591,
   <Cluster: Methymycin / pikromycin pks gene cluster>,
   <Subunit: pikAII pks subunit>,
   <Module: pks module 391>)],
 [(0.159340659340659,
   <Cluster: Lobophorin pks gene cluster>,
   <Subunit: lobA3 pks subunit>,
   <Module: pks module 871>)],
 [(0.149659863945578,
   <Clust